# Exploration

Dans cette partie on va donc explorer nos données afin de se familiariser avec le dataset.

Pour cette exploration j'utilise une variable `version` qui me permet d'avoir le même résultat par rapport à la version du script que je souhaite utiliser.

En interne certaines actions sont effectuées sur le dataset suivant la version. Cela me permet de montrer ma démarche et la progression des traitements / explorations du dataset.

In [ ]:
%load_ext autoreload
%autoreload 2
from src.dataset import Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use(['ggplot', 'https://raw.githubusercontent.com/AlanBlanchet/matplotlib_styles/master/vscode_blue.mplstyle'])

Dataset.init()

On peut faire une cellule de comparaison pour avoir un affichage des données brutes et des données traitées

In [ ]:
Dataset.example("topics1.csv", random_state=0, version=1)

Regardons les mots les plus communs et identifions les mots inutiles

In [ ]:
freq = Dataset.most_common("commons_v0", version=1)
freq.most_common(20)

On remarque dors et déjà qu'on a des ponctuations.

J'ai bien envie de faire un système pour pouvoir voir les phrases en brute et les voir en preprocessé pour voir la différence et comprendre certains cas. On peut donc retirer ces ponctuations puis regarder quelques examples qui nous permettrons d'avancer.

In [ ]:
freq = Dataset.most_common("commons_v0", version=2)
freq.most_common(20)

On remarque déjà qu'il y a le "n't" qui correspond à une négation.

Il y a également des mots qu'on aimerait certainement exclure car il n'apporte par de plus value pour nos prédictions de tags comme par exemple : "problem", "error", "get", "code", "like"...

Pour régler le problème de la négation, je me suis d'abord dit qu'il serait bien de supprimer les chaînes qui ont une longueur égale à 1. Mais rappelons un instant que le langage C comporte 1 seul caractère. Il n'est donc pas possible de procéder comme cela. On aimerait possiblement conserver la négation car cela pourrait être utile pour une phrase du genre : "this is not Java". On gardera dans un premier temps cette négation.

Pour les mots qu'on aimerait exclure, je propose de mettre ces mots dans un fichier texte. Ils feront office de "stopwords" propres au projet. (dans data/exclude.txt)

In [ ]:
Dataset.example("topics1.csv", random_state=0, version=1, interactive=True, index=1)

On a donc tous les outils en main pour commencer à effectuer des analyses plus poussées et effectuer des tests sur nos données.

In [ ]:
df = Dataset.use("topics1.csv", version=0)
df.head()

In [ ]:
sententences_l = df.text.str.len().sort_values()
plt.figure(figsize=(8,5))
plt.title("Text indece by their lengths")
plt.xlabel("Text indice")
plt.ylabel("Text length")
sententences_l.reset_index(drop=True).plot()
sententences_l.describe(percentiles=[0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975])

On remarque que quelques phrases sont très longues (2.5% avec l > 6992)

In [ ]:
target_list = df.target.str.split("|").explode()
print(len(target_list.unique()))

In [ ]:
target_list_counts = target_list.value_counts().sort_values()
target_list_counts_tail = target_list_counts.tail(30)
target_list_counts_tail.plot.barh()
plt.title("Top 30 most commonly used tags in the dataset")
plt.xlabel("Counts")
plt.ylabel("Targets")

In [ ]:
target_list_count_under_5 = target_list_counts[target_list_counts < 5]
target_list_count_under_5

Certains targets ne seront certainement pas simple à trouver car seulement 5000 ont une fréquence de 5 ou +. On peut donc se demander si conserver autant d'éléments est utile. On verra par la suite ce que l'on fait

In [ ]:
target_list_over_5 = target_list[~target_list.isin(target_list_count_under_5.index)]
target_list_over_5_target_count = pd.Series(target_list_over_5.index.value_counts()).value_counts()
target_list_over_5_target_count[0] = len(df) - target_list_over_5_target_count.sum()
plt.figure(figsize=(8,5))
plt.title("Leftover targets number per text\nafter removing low frequency targets (< 5)")
plt.yscale('log')
target_list_over_5_target_count.plot(kind="bar")
plt.xlabel("Number of leftovers")
plt.ylabel("Number of texts")
target_list_over_5_target_count

De plus, si on retirait ces targets, 12 lignes n'auraient plus de target du tout.

# Preprocessing avancé

Dans cette partie on va regarder un peu plus notre dataset pour trouver des éléments mal preprocessé et noter l'index des phrases où l'on trouve ces cas

In [ ]:
Dataset.example("topics1.csv", random_state=0, version=2, interactive=True, index=1)

Voici les élément que nous devons maintenant traiter ainsi que l'index des phrases où le problème a été repéré :

- Les nombres (2, 14)
- Le "#" de C# est coupé (6)
- Attention à ne pas supprimer délaisser des nombres importants comme 64 (8)
- Dates peuvent être importantes (8)
- Le "++" de C++ est coupé (10,30)
- Il y a des urls (12,19,26,34,35,52,54)
- Des morceaux de code sans block de code (12,13,26,27,52,500)
- On supprime les balises "code" ce qui a pour effet de laisser certains textes un peu trop vide (14,19,20)

Constats :
- Certains éléments sont très compliqués à tagger même pour un être humain (9,11,500)
- La négation est importants dans certains contextes (10)
- Certains tags seront quasiement introuvable pour certaines situations. Il faudra habilement choisir une fonction de score (tous les exemples)


## Actions

- Retirer les URLs car ils vont brouiller la target (ex: "http", "https", "www" ...)
- Laisser les nombres car ils peuvent avoir une importance
- Retirer 

En effet, peut être que
- Le découpage d'un URL peut donner une indication.
- Les nombres sont importants

Il faut néanmoins corrigé certains autres problèmes dans l'immédiat comme celui du C++/C#.

Pour ce qui est de prendre en compte le contexte (négation) on verra par la suite.

In [ ]:
chars = ["#", "+"]
np.array([target for target in set(target_list)
 if pd.Series(chars).isin(list(target)).any()])

On remarque que beaucoup de targets ont des version (ex: "c++-32") associées à leur nom. J'ai bien peur que le fait que la version soit présente brouille la prédiction.

On se rappellera d'un possible traitement à effectuer par la suite. Je suppose naivement que l'on ne souhaite pas forcément prédire la version spécifique en fonction d'une question mais plutôt juste le tag initial

In [ ]:
Dataset.example("topics1.csv", random_state=0, version=3, interactive=True, index=6)

In [ ]:
Dataset.example("topics1.csv", random_state=0, version=4, interactive=True, index=12)

La prochaine étape serait de s'intéresser à la balise code afin de rajouter de l'information.
En effet, comme on l'a vu précédemment, certaines questions on vu la majorité de contenu se faire supprimer car la majorité était du code.

On peut commencer par essayer, en fonction de la balise de code, de détecter le language qui y est présent.

Après expérimentation avec les packages [guesslang](https://github.com/yoeo/guesslang) / [whats_that_code](https://github.com/matthewdeanmartin/whats_that_code) c'était très difficile. Le seule package qui m'apporte un peu d'espoir est [pygments](https://github.com/pygments/pygments) mais il a parfois de mauvaises prédictions.

Je propose malgré tout de l'utiliser et de voir ce que cela donne.

On va donc simplement remplacer la balise de code par le nom du langage détecté pour chaque morceaux de code.

In [ ]:
Dataset.example("topics1.csv", random_state=0, version=4, interactive=True, index=14)

D'après ce que je vois, ce n'est pas fameux. On pourra revenir sur cette partie plus tard.